### Katarina - Intäkter per månad/ vecka

In [ ]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [ ]:
import pandas as pd
import metrics as M
import viz as V
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("../data/ecommerce_sales.csv", parse_dates=["date"])

In [ ]:
print("Intäkter per månad:")
display(M.revenue_month(df))

In [ ]:
print("Intäkter per vecka (sorterad):")
display(M.sorted_weeks(df))

In [ ]:
print("Graf för intäkt per månad:")
V.plot_revenue_month(df)

print("Graf för intäkt per vecka:")
V.plot_revenue_week(df)

Magdalena - Försäljning per stad

In [ ]:
by_city = M.revenue_per_city(df)
by_cat = M.top_categorys(df)
by_city
by_cat.head(3) # Top 3 kategorier

In [ ]:
# Intäkt per stad
fig, ax = plt.subplots(figsize=(10, 6)) # skapar en graf med sepererade staplar för varje stad
V.bar(ax, by_city["city"], by_city["revenue"], "Intäkt per stad", "Stad", "Intäkt i miljoner")

# Intäkt per kategori
fig, ax = plt.subplots(figsize=(10, 6)) # skapar en graf med sepererade staplar för varje stad
V.bar(ax, by_cat["category"], by_cat["revenue"], "Intäkt per kategori", "Kategori", "Intäkt i miljoner")



# Kevin - AOV

In [ ]:

df = pd.read_csv("../data/ecommerce_sales.csv")

np_orders = np.array(df.revenue)

q1 = np.percentile(np_orders, 25)
q3 = np.percentile(np_orders, 75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

filtered_orders = np_orders[(np_orders >= lower_bound) & (np_orders <= upper_bound)]

std = np.std(filtered_orders)

mean = np.mean(filtered_orders)

plt.figure(figsize=(8, 5))
plt.hist(filtered_orders, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(mean, color='red', linestyle='dashed', linewidth=2, label=f"Mean = {mean:.2f}")
plt.axvline(mean + std, color='green', linestyle='dotted', linewidth=2, label=f"+1 STD = {mean + std:.2f}")
plt.axvline(mean - std, color='green', linestyle='dotted', linewidth=2, label=f"-1 STD = {mean - std:.2f}")

plt.title("Revenue Distribution (Outliers Removed)")
plt.xlabel("Revenue")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

## Rekomendationer när det gäller intäkter per månad/vecka:

- Tänk på en stark marknadsföring under januari och maj. Dessa månader visar redan höga intäkter och har potential att generera ännu större omsättning.

- Fundera över åtgärder för att motverka nedgången i mars, (kanske kampanjer eller rabatter) för att jämna ut intäktsnivåerna. 

- Fortsätt analysera och jobba med veckoresultaten för att få fram och identifera återkommande veckor med toppar/dalar och anpassa bemanning och lager därefter för maximal effektivitet och besparing. 

## Jan - Eventuella Avvikelser:


In [ ]:
print("Per stad:")
revenue_per_city, median_revenue = M.revenue_per_city_dev(df)
display(revenue_per_city)
display(median_revenue)
V.plot_city_revenue(revenue_per_city, median_revenue)
#Markerat städer som sticker ut mer än 30% från genomsnittet(Låg och hög)

In [ ]:
df["date"] = pd.to_datetime(df["date"])  # säkerställ att det är datetime-format
df["month"] = df["date"].dt.to_period("M").astype(str)

df = M.add_month_column(df)
grouped = M.sales_per_cat(df, calc_change=True)

print(grouped)
#Visar försäjlning per kategori per månad och om försäjlningen minskar eller ökar i %.

In [ ]:

df["date"] = pd.to_datetime(df["date"])  # säkerställ att det är datetime-format
df["month"] = df["date"].dt.to_period("M").astype(str)
sales = M.sales_per_cat(df, calc_change=True)
sales = M.detect_sales_anomalies(sales)

df = M.sales_per_cat(df)
sales_outliers= sales[sales["anomaly"] != "Normal"] #visar de kategorier som har ökat eller minskat kraftigt(månadsvis,jamfört med månaden innan)
print(sales_outliers)


In [ ]:
df = pd.read_csv("../data/ecommerce_sales.csv")
antal_hög_unite, antal_låg_unite, df_large_orders = M.order_controll(df, column="units")

print(f"Antal order med hög unit värde: {antal_hög_unite}")
print(f"Antal order med låg unit värde: {antal_låg_unite}")
print("Ordrarna med väldigt många units:")
display(df_large_orders)

In [ ]:
mean_price, antal_små_ordar = M.price_order(df, column="price")
print(f"Genomsnittet av kategorien price: {mean_price}")
print(f"Antalet ordar med pris som ligger under genomsnittet: {antal_små_ordar}")


In [ ]:
median_revenue, liten_revenue, count_avg = M.revenue_order(df, column="revenue")



df_little_revenue = df[df["revenue"] < 100]
#Visar en genomsnitt av revenue samt de som har mindre än genomsnitt och de som är extrem låga (100)
print(f"Medianen av revenue {median_revenue}")
print(f"Antalet ordar som är mindre än medianen: {count_avg}")
print("Ordrar som har extremt låg revenue: ")
print(df_little_revenue)

In [ ]:
df["date"] = pd.to_datetime(df["date"])  # säkerställ att det är datetime-format
df["month"] = df["date"].dt.to_period("M")

monthly_revenue = M.month_revenue(df, column="month")
print(f"Revenue per månad {monthly_revenue}")

V.plot_monthly_revenue(monthly_revenue)


## Jan - Observationer:
-  Vid analys av omsättning per stad ser man tydligt att Stockholm ligger över 30% över genomsnittet. Göteborg, Malmö och Uppsala ligger inom den vanliga gränsen. Västerås har en negativ/låg avvikelse med mer än 30% mindre än genomsnittet.
- Analysen visar tydligt hur efterfrågan varierar mellan de olika kategorierna. Kategorien "Beauty" minskar krafitgt i februari (-52%) men ökar kraftigt i Mars (+68%). Electronics kategorien visar en stor nedgång i Mars med (-41%) medan Clothing och Sports kategorien ökar kraftigt under våren.
- Analysen visar att några ordrar har ovanligt hög units värde (>7), vilket gör dem tyligt större än genomsnittet. Det finns flera ordar med mycket låg omsättning,ofta under 100 kr jamfört med medianen på cirka 1089 kr. Priserna i sig verkar stabila mellan kategorierna.
- I analysen av "Revenue per månad" hittar man inga konsithgeter dvs inga hopp eller extrem stora skillnader mellan månaderna dock så är revenuen lägst i månaden Mars 